In [1]:
!pip install git+https://github.com/MagedSaeed/farasapy.git pyarabic Tashaphyne

  Cloning https://github.com/MagedSaeed/farasapy.git to /tmp/pip-req-build-mtwwxh0g
  Running command git clone --filter=blob:none --quiet https://github.com/MagedSaeed/farasapy.git /tmp/pip-req-build-mtwwxh0g
  Resolved https://github.com/MagedSaeed/farasapy.git to commit 9200fb78f9e4d102a4b63b54b9b0ace84ff9d43e
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.5/251.5 kB 6.4 MB/s eta 0:00:0000:01
  Created wheel for farasapy: filename=farasapy-0.0.14-py3-none-any.whl size=12920 sha256=a8e0e09d63926179501cbf7ef27d3518ea723aa04d795f477d458cacb58393d2
  Stored in directory: /tmp/pip-ephem-wheel-cache-qsjo9h73/wheels/ae/56/72/2ba224ca1aa5d0822d5b1ea26be589d58732b74a6dd2a04f62
Successfully built farasapy


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import pandas as pd
from transformers import AdamW
from tqdm import tqdm
from transformers import get_scheduler
import re
import regex
from nltk.stem.isri import ISRIStemmer
from nltk.stem.snowball import SnowballStemmer
from pyarabic.araby import strip_tashkeel, strip_diacritics
import numpy as np
import pandas as pd
import pyarabic.araby as araby
from sklearn.feature_extraction.text import TfidfVectorizer
from farasa.stemmer import FarasaStemmer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tashaphyne.stemming import ArabicLightStemmer

In [3]:
train_set = pd.read_csv('/kaggle/input/c/detect-llm-generated-arabic-text/train_set.csv')
dev_set = pd.read_csv('/kaggle/input/c/detect-llm-generated-arabic-text/dev_set.csv')
test_set = pd.read_csv('/kaggle/input/c/detect-llm-generated-arabic-text/test_set.csv')

In [4]:
def remove_stop_words(text):
    Text=[i for i in str(text).split() if i not in arabic_stopwords]
    return " ".join(Text)

def ISRI_Stemmer(text):
    #making an object
    stemmer = ISRIStemmer()

    #stemming each word
    text = stemmer.stem(text)
    text = stemmer.pre32(text)
    text = stemmer.suf32(text)

    return text

def Snowball_stemmer(text):
    text = text.split()
    #making an object
    stemmer = SnowballStemmer("arabic")

    #stemming each word
    text=[stemmer.stem(y) for y in text]

    return " " .join(text)

def Arabic_Light_Stemmer(text):
    #making an object
    Arabic_Stemmer = ArabicLightStemmer()

    #stemming each word
    text=[Arabic_Stemmer.light_stem(y) for y in text.split()]

    return " " .join(text)

def normalizeArabic(text):
    text = text.strip()
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)

    #remove repetetions
    text = re.sub("[إأٱآا]", "ا", text)
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('ييي', 'ي')
    text = text.replace('اا', 'ا')

    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)

    # Remove longation
    text = re.sub(r'(.)\1+', r"\1\1", text)

    #Strip vowels from a text, include Shadda.
    text = araby.strip_tashkeel(text)

    #Strip diacritics from a text, include harakats and small lettres The striped marks are
    text = araby.strip_diacritics(text)
    text=''.join([i for i in text if not i.isdigit()])
    return text

def Removing_non_arabic(text):
    text = re.sub('[A-Za-z]+',' ',text)
    return text

def Removing_numbers(text):
    text=''.join([i for i in text if not i.isdigit()])
    return text

def Removing_punctuations(text):
    ## Remove punctuations
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
    text = text.replace('؛',"", )

    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)
    text =  " ".join(text.split())
    return text.strip()

def Removing_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)
def remove_extra_Space(text):
    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)
    return  " ".join(text.split())

def remove_small_sentences(df):
    for i in range(len(df)):
        if len(df.text.iloc[i].split()) < 3:
            df.text.iloc[i] = np.nan
def apply_farasa_stemming(text):
    """
    Applique le stemming à un texte en arabe en utilisant Farasa.

    Args:
        text (str): Le texte en arabe à traiter.

    Returns:
        str: Le texte après application du stemming.
    """
    # Initialiser le stemmer
    stemmer = FarasaStemmer()

    # Appliquer le stemming
    stemmed_text = stemmer.stem(text)

    return stemmed_text
def preprocess_text(data):
    """
    Preprocess text data by applying a series of cleaning functions.

    Parameters:
        data (pd.DataFrame): DataFrame containing the text data.
        stopword_removal_func (function): Function to remove stopwords.
        non_arabic_removal_func (function): Function to remove non-Arabic characters.
        normalization_func (function): Function to normalize Arabic text.
        number_removal_func (function): Function to remove numbers.
        punctuation_removal_func (function): Function to remove punctuations.

    Returns:
        pd.DataFrame: Cleaned DataFrame.
    """
    print('remove stop words')
    data['text'] = data['text'].apply(lambda text: remove_stop_words(text))
    print('remove non arabic')
    data['text'] = data['text'].apply(lambda text: Removing_non_arabic(text))
    print('normalize arabic')
    data['text'] = data['text'].apply(lambda text: normalizeArabic(text))
    print('remove numbers')
    data['text'] = data['text'].apply(lambda text: Removing_numbers(text))
    print('remove punctuations')
    data['text'] = data['text'].apply(lambda text: Removing_punctuations(text))
    # print('stemming')
    # data['text'] = data['text'].apply(lambda text: Arabic_Light_Stemmer(text))
    return data

In [5]:
import nltk
from nltk.corpus import stopwords # import stopwords

nltk.download('stopwords')
#are there any stop words in the data?
arabic_stopwords = stopwords.words("arabic")
len(arabic_stopwords)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


754

In [6]:
train_set = preprocess_text(train_set)
#train_set['text'] = train_set['text'].apply(preprocess_text)
train_set.head()

remove stop words
remove non arabic
normalize arabic
remove numbers
remove punctuations


,text_id,text,generated
0,9519,ارض السوار وكيف ظل البحاره يرفعون الاحمال والا...,0
1,6247,وقال كيه لحسان » قل فوالله لقولك اشد عليهم وقع...,0
2,3065,غابه ناءيه عميقه تسكن الاشجار الضخمه والنباتات...,1
3,652,صميم قريه العتمه تتجذر اشجار الزيتون العتيقه ت...,1
4,3447,قلب مدينه مزدحمه تنم يوما تتشابك الانوار الظلا...,1


In [7]:
dev_set = preprocess_text(dev_set)
# dev_set['text'] = dev_set['text'].apply(preprocess_text)
dev_set.head()

remove stop words
remove non arabic
normalize arabic
remove numbers
remove punctuations


,text_id,text,generated
0,2650,لحظه غامضه عندما انحنت الشمس الافق واخذت احضان...,1
1,8091,وارسطو تكوين الطباع والكمال الحلقي للانسان » ا...,0
2,3709,قلب المدينه القديمه تختلط الرواءح العتيقه والح...,1
3,10437,غاليه البقميه حصنا منيعا السقوط المدوي ظلت اصد...,0
4,3577,الجبال الشماليه كانت الرياح تعصف بالاشجار المي...,1


In [8]:
test_set = preprocess_text(test_set)
# test_set['text'] = test_set['text'].apply(preprocess_text)
test_set.head()

remove stop words
remove non arabic
normalize arabic
remove numbers
remove punctuations


,text_id,text
0,2227,صحراء الجزيره العربيه الواسعه تتراقص الكثبان ا...
1,6682,السواد سواد الشعر مولاناء مو دليل تعال شوف شنو...
2,2649,وضعت قدميك حافه الرصيف تتراقص ظلال الاشجار الب...
3,6652,وقف رجال الشراكسه صامتين ازاء هذاء وقد عقدوا ا...
4,4296,انبلاج الفجر الافق ارتسمت اولي خيوط الشمس الحق...


In [9]:
import torch

# Vérifier si un GPU est disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation du périphérique: {device}")

Utilisation du périphérique: cuda


In [10]:
from transformers import BertTokenizer, BertModel

# Charger le tokenizer et le modèle BERT
tokenizer = BertTokenizer.from_pretrained("asafaya/bert-base-arabic")
model = BertModel.from_pretrained("asafaya/bert-base-arabic").to(device)  # Déplacer le modèle sur le GPU

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/334k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [11]:
def encode_text(text, tokenizer, max_length=350):
    encoded = tokenizer.encode_plus(
        text,
        max_length=max_length,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    # Déplacer vers le GPU
    return encoded["input_ids"].to(device), encoded["attention_mask"].to(device)

def get_bert_embeddings(text, tokenizer, model):
    input_ids, attention_mask = encode_text(text, tokenizer)
    
    # Passer les données sur le GPU et exécuter le modèle sans calcul des gradients
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        embeddings = outputs.last_hidden_state
        
    # Retourner les embeddings du [CLS] token
    return embeddings[:, 0, :].cpu().numpy()  # Déplacer vers le CPU avant de convertir en numpy


In [12]:
# Appliquer l'encodage et obtenir les embeddings pour chaque texte
train_set['embedding'] = train_set['text'].apply(lambda x: get_bert_embeddings(x, tokenizer, model))

In [13]:
# Appliquer l'encodage et obtenir les embeddings pour chaque texte
dev_set['embedding'] = dev_set['text'].apply(lambda x: get_bert_embeddings(x, tokenizer, model))

In [14]:
# Appliquer l'encodage et obtenir les embeddings pour chaque texte
test_set['embedding'] = test_set['text'].apply(lambda x: get_bert_embeddings(x, tokenizer, model))

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Extraire les embeddings et les étiquettes
X_train = list(train_set['embedding'])  # Embeddings
y_train = train_set['generated'].values  # Labels

# Aplatir les embeddings
X_train = [embedding.flatten() for embedding in X_train]

# Diviser en données d'entraînement et de test
X_test = list(dev_set['embedding'])
y_test = dev_set['generated'].values  # Labels

# Aplatir les embeddings
X_test = [embedding.flatten() for embedding in X_test]


X_submission = list(test_set['embedding'])  # Embeddings
X_submission = [embedding.flatten() for embedding in X_submission]

# Normaliser les embeddings
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_submission = scaler.transform(X_submission)



In [16]:
_ = '''
from sklearn.model_selection import GridSearchCV

# Paramètres à tester
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.001, 0.01, 0.1, 1],
    'kernel': ['rbf', 'linear']
}

# Recherche par grille
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Affichage des meilleurs paramètres et du score
print(f"Meilleurs paramètres: {grid_search.best_params_}")
print(f"Meilleur score: {grid_search.best_score_}")
'''


In [17]:
# Récupérer les meilleurs paramètres du grid search
# best_params = grid_search.best_params_

# Créer et entraîner le modèle SVM avec les meilleurs paramètres
svm_best = SVC(C=10, gamma=0.001, kernel='rbf')
svm_best.fit(X_train, y_train)

SVC(C=10, gamma=0.001)

In [18]:
# Predict on development set
y_dev_pred_rf_model = svm_best.predict(X_test)

In [19]:
# Evaluate on development set
print("Development Set Evaluation:")
print(f"Accuracy: {accuracy_score(y_test, y_dev_pred_rf_model)}")
print("Classification Report:")
print(classification_report(y_test, y_dev_pred_rf_model))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_dev_pred_rf_model))

Development Set Evaluation:
Accuracy: 0.9979310344827587
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       707
           1       1.00      1.00      1.00       743

    accuracy                           1.00      1450
   macro avg       1.00      1.00      1.00      1450
weighted avg       1.00      1.00      1.00      1450

Confusion Matrix:
[[707   0]
 [  3 740]]


In [20]:
import joblib
# Sauvegarder le modèle SVM dans un fichier
joblib.dump(svm_best, 'svm_model_V2.pkl')

['svm_model_V2.pkl']

In [21]:
y_test_pred = svm_best.predict(X_submission)

In [22]:
# Create a DataFrame with the 'id' and predicted labels
results_df = pd.DataFrame({
    'text_id': test_set['text_id'],
    'generated': y_test_pred
})

# Save the DataFrame to a CSV file
results_df.to_csv('submission.csv', index=False)

In [23]:
results_df

,text_id,generated
0,2227,1
1,6682,0
2,2649,1
3,6652,0
4,4296,1
...,...,...
1446,8450,0
1447,6758,0
1448,6452,0
1449,5764,0


In [24]:
results_df['generated'].value_counts()

generated
1    744
0    707
Name: count, dtype: int64